In [1]:
# import torch
# from torch.autograd import Variable
from sparse_identification.utils import derivative 
import pandas as pd

#--> Standard python library.
%pylab

#--> Matplotlib-related.
%matplotlib inline
from matplotlib.gridspec import GridSpec
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D

# params = {'text.usetex' : True,
#           'font.size' : 8*2,
#           'font.family' : 'lmodern',
#           'text.latex.unicode' : True}
# plt.rcParams['text.latex.preamble']=[r'\usepackage{lmodern}', r'\usepackage{bm}']
# plt.rcParams.update(params)
plt.style.use('seaborn-muted')
w = 10 # plot figure width
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell
    text-align: right;
    vertical-align: middle;
}
</style>
""")
# from latex_envs.latex_envs import figcaption

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


\title{A (very) quick introduction to SINDy}
\author{Kevin, Joel, Elbert$^1$}
\maketitle
$^1$Harvard University
<center>**Abstract**</center>
After a quick presentation of the training and testing datasets used, the system identification capabilities of SINDy are illustrated using time-series of the Lorenz system. Four different algorithms for system identification have been implemented so far and will be compared:
- Ordinary Least-Squares
- Sequentially hard-thresholded ordinary least-squares
- $\ell_1$-penalized least-squares.

Wait that's only 3 though\\

\begin{equation}
    \begin{aligned}
        & \dot{x} = \sigma (y-x) \\
        & \dot{y} = x(\rho -z) - y \\
        & \dot{z} = xy - \beta z
    \end{aligned}
    \label{eq: Lorenz system}
\end{equation}
Lorenz example $\sigma = 10$, $\beta = \displaystyle \frac{8}{3}$ and $\rho=28$.

In [2]:
import h5py, numpy as np
a=h5py.File('C:/Users/owner/Desktop/Spring18/RNN-dynamics/threebit_flipflop/data/net4_test.mat','r')
a.keys()

IOError: Unable to open file (File signature not found)

In [ ]:
a['outData'][0]

In [ ]:
import sys
sys.path

In [ ]:
class PCA():
    def __init__(self, list_of_matrices, nmodes):
        orig_dim = list_of_matrices[0].shape[1]
        assert(all(matrix.shape[1]==orig_dim for matrix in list_of_matrices))
        X = np.concatenate(list_of_matrices, 1)
        U, s, VT = np.linalg.svd(X)
        self.param_dict = {'U':U,'s':s,'VT':VT}
        self.nmodes = nmodes
        # np.matmul(U, np.matmul(S,VT[:,:modes]))
    def forward(self,matrix):
        return np.matmul(matrix, self.param_dict['VT'][:,:self.nmodes])
        
    def plot_SVs(self):
        # S = np.zeros([self.param_dict['U'].shape[0],self.param_dict['VT'].shape[0]])
        # S[0:nmodes,0:nmodes] = np.eye(nmodes)
        plt.plot(np.log(self.param_dict['s']))
        plt.show()
        
def PCA_project(X, modes, plot_SVs=True):
    # SVD of X
    U, s, VT = np.linalg.svd(X)
    
    # construct S matrix with S_vector -- there must be a better way to do this?
    S = np.zeros([U.shape[0],VT.shape[0]])
    for i in range(modes):
        S[i,i] = s[i]

#     print("shapes", X.shape, U.shape,S.shape,VT[:,:modes].shape, (U@S@VT[:,:modes]).shape)
    plt.plot(np.log(s))
    plt.show()
    return np.matmul(U, np.matmul(S,VT[:,:modes]))

In [ ]:
# --> Simulating the Lorenz system.
# from Lorenz import Lorenz
# --> Sets the parameters to their classical values.
# sigma, rho, beta = 10., 28., 8./3.

# --> Integration time.
t = np.linspace(0, 39, 40)
assert(t[1]-t[0]==1)
# Lorenz.py assumes the times are in increments of 1
# 0,20,100

# --> Produce the date to be used in the sparse identification.
# x0 = np.array([-1, -1, -1]) # Initial condition.
# x, dx = Lorenz(x0, sigma, rho, beta, t)

fulldata = pd.read_csv("../../../rnndata/no_inputs.csv", header = None).values
x = PCA_project(fulldata, 3, False)
# inputs = pd.read_csv("../../../rnndata/worldtour_inputs.csv", header = None).values # 4-22 new
# x = np.hstack([x,inputs])
dx = np.gradient(x)
dx = dx[0]

# Other way:
# pca = PCA([fulldata],3)
# x = pca.forward(fulldata) # and make dx as above
# equivalently, dx = pca.forward(np.gradient(fulldata)[0])
# not equivalent to running a whole new SVD on np.gradient(fulldata)[0] 


# use first element of dx, dx[0], to return time derivative dx/dt 
# dx0 = derivative(X, t[1]-t[0])
# --> Slightly different initial condition to highlight the chaotic nature. # hey we should try this too!
# y0 = np.array([-8.01, 7., 27.]) # Initial condition.
# y, dy = Lorenz(y0, sigma, rho, beta, t)

# They cheated with their initial conditions. What should our IC be?
y0 = x[0] # need this for last step odeint

In [ ]:
x.shape

In [ ]:
# --> Plot time traces of the two trajectories as well as the corresponding strange attractor.
fig = plt.figure(figsize=(1.5*w, w/2))
gs = GridSpec(3, 6)

ax0 = fig.add_subplot(gs[0, :3])
ax0.plot(t, x[:, 0])
# ax0.plot(t, y[:, 0])
ax0.set_ylabel('x')
ax0.set_xticks([])
ax0.set_xlim(0, 320)

ax1 = fig.add_subplot(gs[1, :3])
ax1.plot(t, x[:, 1])
# ax1.plot(t, y[:, 1])
ax1.set_ylabel('y')
ax1.set_xticks([])
ax1.set_xlim(0, 320)

ax2 = fig.add_subplot(gs[2, :3])
ax2.plot(t, x[:, 2])
# ax2.plot(t, y[:, 2])
ax2.set_ylabel('z')
ax2.set_xlabel('t')
ax2.set_xlim(0, 320)

ax3 = fig.add_subplot(gs[:, 3:], projection='3d')
ax3.plot(x[:, 0], x[:, 1], x[:, 2])
# ax3.plot(y[:, 0], y[:, 1], y[:, 2])
ax3.set_xlabel('x', labelpad=10)
ax3.set_ylabel('y')
ax3.set_zlabel('z')


Does our thing have sensitivity to initial conditions? \cite{book:sparrow:2012}

\section{Identification}

In the following, we will compare the capabilities of different algorithms used to solve the system identification problem presented earlier. For that purpose, the dictionary of functions considered is given by

$$
    \Theta({\bf x}) = \begin{bmatrix} 1 & {\bf x} & P_2({\bf x}) & P_3({\bf x}) & P_4({\bf x}) & P_5({\bf x}) \end{bmatrix}
$$

where $P_i({\bf x})$ is a dictionary consisting of all the i-th order polynomials in the entries of ${\bf x}$. For $i=2$, this dictionary is thus given by

$$
    P_2({\bf x}) = \begin{bmatrix} x^2 & xy & xz & y^2 & yz & z^2\end{bmatrix}
$$

The scikit-learn package provides useful utilities functions to construct such dictionaries of functions.

In [ ]:
# --> Creation of the library Theta.
from sklearn.preprocessing import PolynomialFeatures
library = PolynomialFeatures(degree=5, include_bias=True) # too scared
# library.get_feature_names() or get_params()
Theta = library.fit_transform(x) # timesteps (320) by num_features (56) (or 59 lol)
# Theta = np.hstack([Theta,inputs]) # modified here to throw in inputs
n_lib = library.n_output_features_ # +3
# Ok I'm basically tacking on the inputs as a linear term. But that's a simplification
# I could throw them in earlier so sindy tries nonlinear terms with inputs in them

from scipy.linalg import block_diag
A = block_diag(Theta, Theta, Theta) # 3x(320 by 56)
b = dx.flatten(order='F')[:len(t)*3] # 960

In [ ]:
print(dir(library))
# with inputs, library.get_feature_names() etc are wrong
Theta.shape,A.shape,dx.shape,b.shape

\subsection{Ordinary least squares}

The ordinary least squares is probably the simplest approach existing for system identification. The corresponding minimization problem simply reads

\begin{equation}
    \min_{\xi_{OLS}} \| \Theta({\bf x})\xi_{OLS} - \dot{\bf x} \|_2^2
\end{equation}

In this case, the least-squares solution vector $\xi_{OLS}$ can very easily be obtained. Note however that, being unconstrained, this solution vector will be dense, i.e. all of its coefficients are non-zero.

In [ ]:
import scipy.io
a = scipy.io.loadmat('../../../../RNN-dynamics/threebit_flipflop/data/net4_test.mat')

In [ ]:
a.keys()

In [ ]:
%%time
# --> Least-squares estimator.
from sparse_identification import sindy
ols = sindy(l1=0, l2=0, solver='lstsq') # only 'lstsq' and 'lasso' are implemented

# --> Fit the OLS model.
ols.fit(A, b)
print 'Total number of possible terms :', ols.coef_.size
print 'Number of non-zero coefficients :', np.count_nonzero(ols.coef_)

In [ ]:
# --> Print on-screen the coefficients of the OLS estimator.

print "Identified equation for x : \n"
print ols.coef_[:n_lib], "\n"
print "\n Identified equation for y : \n"
print ols.coef_[n_lib:2*n_lib], "\n"
print "\n Identified equation for z : \n"
print ols.coef_[2*n_lib:3*n_lib]

As shown, all of the coefficients of the solution $\xi_{OLS}$ are effectively non-zero. From a mathematical point of view, this denseness of $\xi_{OLS}$ indicates that the ordinary least-squares estimator uses absolutely all of the atoms included in our input dictionary $\Theta({\bf x})$ to approximate the true function $\mathcal{f}({\bf x})$. Given the simplicity of the Lorenz system, this behavior is clearly related to *overfitting*, a well-known problem of regression. Note moreover that, when simulating the identified model, the inclusion of all the extra terms may slow down the computation or even cause numerical instabilities.

\subsection{$\ell_1$-penalized least squares}

A good heuristic to obtain a sparse solution vector $\xi$ is to replace the nuclear norm $\| \xi \|_0$ by its 1-norm $\| \xi \|_1$ in the original problem. The resulting minimization problem reads

\begin{equation}
        \min_{\xi} \| \Theta({\bf x})\xi - \dot{\bf x} \|_2^2 \\
        \text{s. t. }  \| \xi \|_1 \le \alpha
\end{equation}

where $\alpha$ is typically chosen as a fraction of $\| \xi_{OLS} \|_1$. This problem is a convex minimization problem which can easily be solved using standard convex optimization libraries. Note however that, although the resulting solution vector $\xi_{\ell_1}$ is sparse, it is usually not the optimal solution. Hence, once the sparsity pattern of $\xi$ has been determined, one then runs an additional ordinary least-squares regression retaining only the atoms of $\Theta({\bf x})$ corresponding to non-zero entries of $\xi_{\ell_1}$.

In [ ]:
%%time
# --> l1-penalized estimator.
from sparse_identification import sindy
l1_penalized = sindy(l1=0.1, solver='lasso') # originally l1=0.01

# --> Fit the OLS model.
l1_penalized.fit(A, b)
print 'Total number of possible terms :', l1_penalized.coef_.size
print 'Number of non-zero coefficients :', np.count_nonzero(l1_penalized.coef_)

In [ ]:
# --> Print on-screen the coefficients of the l1-penalized estimator.

print "Identified equation for x : \n"
print l1_penalized.coef_[:n_lib], "\n"
print "\n Identified equation for y : \n"
print l1_penalized.coef_[n_lib:2*n_lib], "\n"
print "\n Identified equation for z : \n"
print l1_penalized.coef_[2*n_lib:3*n_lib]

The $\ell_1$-penalized estimator retains only 7 atoms out of the 168 available ones in $\Theta({\bf x})$. As will be shown, discarding these 161 extra atoms hardly influences the accuracy of the fit. Moreover, the coefficients identified are very close to the actual ones used in the definition of system \eqref{eq: Lorenz system}.

\subsection{Sequentially hard-thresholded ordinary least squares}

The last algorithm considered is the one presented in \cite{pnas:brunton:2016}. It is an iteratively hard-thresholded least-squares approach. At each iteration, only the atoms of $\Theta({\bf x})$ for which

$$
    \vert \xi_i \vert > \ell_1 \displaystyle \frac{1}{n}\sum_{j=1}^n \vert \xi_j \vert
$$

are retained. Note that only the non-zero entries of $\xi$ are considered when evaluating the mean absolute value.

In [ ]:
%%time
# --> Sequentially hard-thresholded estimator.
from sparse_identification import sindy
shols = sindy(l1=0.3, solver='lstsq')

# --> Fit the OLS model.
shols.fit(A, b)
print 'Total number of possible terms :', shols.coef_.size
print 'Number of non-zero coefficients :', np.count_nonzero(shols.coef_)

In [ ]:
# --> Print on-screen the coefficients of the SH-OLS estimator.

print "Identified equation for x : \n"
print shols.coef_[:n_lib], "\n"
print "\n Identified equation for y : \n"
print shols.coef_[n_lib:2*n_lib], "\n"
print "\n Identified equation for z : \n"
print shols.coef_[2*n_lib:3*n_lib]

The sparsity patterns of the solution vector $\xi$ obtained using the $\ell_1$-penalized least-squares estimator or the present sequentially hard-thresholded one are identical. Not however that may not always be true, although they usually marginally differ from one another.

\subsection{Comparison of the predicted dynamics}

In [ ]:
# from Lorenz import Identified_Model # function actually ignores t, because system is autonomous
# inputs: y,t,library,ols. output: dy
# maybe i could have Identified_Model take in inputs (bit flips)... but how do they affect dy? let sindy figure it out 
# x_thing = Identified_Model(y0,4,library,ols) 
# x_thing
# ols.coef_
# np.hstack([library.transform(np.zeros((1,3))),inputs[2:3,:]])

In [ ]:
# def map_out(model,y0,t,library,ols,inputs):
#     ys = np.zeros((len(t),3))
#     ys[0] = y0
#     for i in range(len(t)-1):
#         dy = model(ys[i],t[i],library,ols,inputs[i])
#         ys[i+1] = ys[i]+dy
#         print(dy,ys[i+1])
        
# x_ols = map_out(Identified_Model,y0,t,library,ols,inputs)
# x_shols = map_out(Identified_Model,y0,t,library,shols,inputs)
# # doesn't work (numerical instability)

In [ ]:
Identified_Model(y0,t[0],library,shols)

In [ ]:
# --> Run the different identified models.
from scipy.integrate import odeint
from Lorenz import Identified_Model
# function that computes the derivative of output given y, t0, args are additional arguments to Identified_Model
x_ols = odeint(Identified_Model, y0, t, args=(library, ols))
x_shols = odeint(Identified_Model, y0, t, args=(library, shols))


# --> Plot the results.
fig = plt.figure(figsize=(1.5*w, w/2))
gs = GridSpec(3, 6)

ax0 = fig.add_subplot(gs[0, :3])
ax0.plot(t, x[:, 0])
ax0.plot(t, x_ols[:, 0])
ax0.plot(t, x_shols[:, 0])
ax0.set_ylabel('x')
ax0.set_xticks([])
# ax0.set_xlim(0, 20)

ax1 = fig.add_subplot(gs[1, :3])
ax1.plot(t, x[:, 1])
ax1.plot(t, x_ols[:, 1])
ax1.plot(t, x_shols[:, 1])
ax1.set_ylabel('y')
ax1.set_xticks([])
# ax1.set_xlim(0, 20)

ax2 = fig.add_subplot(gs[2, :3])
ax2.plot(t, x[:, 2])
ax2.plot(t, x_ols[:, 2])
ax2.plot(t, x_shols[:, 2])
ax2.set_ylabel('z')
ax2.set_xlabel('t')
# ax2.set_xlim(0, 20)

ax3 = fig.add_subplot(gs[:, 3:], projection='3d')
ax3.plot(x[:, 0], x[:, 1], x[:, 2])
ax3.plot(x_ols[:, 0], x_ols[:, 1], x_ols[:, 2])
ax3.plot(x_shols[:, 0], x_shols[:, 1], x_shols[:, 2])
ax3.set_xlabel('x', labelpad=10)
ax3.set_ylabel('y')
ax3.set_zlabel('z')
ax3.locator_params(axis='x', nbins=5)

# figcaption('(Left) Time-traces of the different state variables and (right) corresponding trajectories in the three-dimensional phase space. \
#             In all figures, blue lines denote the ground truth evolution, while green and red are the predictions of the OLS and SH-OLS models.')

# References

[<a id="cit-book:sparrow:2012" href="#call-book:sparrow:2012">1</a>] C. Sparrow, ``_The Lorenz equations: bifurcations, chaos, and strange attractors_'',  2012.

[<a id="cit-pnas:brunton:2016" href="#call-pnas:brunton:2016">2</a>] Brunton Steven L, Proctor Joshua L and Kutz J Nathan, ``_Discovering governing equations from data by sparse identification of nonlinear dynamical systems_'', Proceedings of the National Academy of Sciences, vol. 113, number 15, pp. 3932--3937,  2016.



In [ ]:
libz = library.fit_transform(y0.reshape(1,-1)) # 1,56
inpz = np.zeros((1,3))
libz = np.hstack([libz,inpz]) # 1,59. this assumes inputs[t] corresponds to time t (not true for small time increments)
print(libz.shape)

What a struggle! I'm still tryna figure out this whole SINDy with inputs business. Ideas:
-write a new library function for inputs. i'd need to do this for trigonometric features.
-just bite the bullet and make inputs "variables". this will explode the number of parameters, but whatever. the problem is, this causes the matrix A to not have sufficient rank or something.
-why in odeint are the times not going to 320? there seems to be numerical instability around time 23 or something
-why does shols time integration start from 1304 something?
-did i choose the right initial condition? can i shrink dt?
-maybe try a sparser or thicker system?

Why are we doing SINDy? We're taking a big NLDS (1000), making it a small NLDS (3\*56 but with sparsity).